## <font color='darkblue'>Preface</font>
([article source](https://towardsdatascience.com/making-interactive-visualizations-with-python-altair-7880ab5cf894)) <font size='3ptx'>**A comprehensive practical guide**</font>

Data visualization is a fundamental piece of data science. If used in exploratory data analysis, **data visualizations are highly effective at unveiling the underlying structure within a dataset or discovering relationships among variables.**

Another common use case of data visualizations is to deliver results or findings. They carry much more informative power than plain numbers. Thus, we **often use data visualization in storytelling, a critical part of the data science pipeline.**

We can enhance the capabilities of data visualizations by adding interactivity. **The [Altair library](https://altair-viz.github.io/) for Python is highly efficient at creating interactive visualizations.**

**In this article, we will go over the basic components of interactivity in Altair. We will also do examples to put these components into action.** Let’s start by importing the libraries.

In [2]:
#!pip install altair

In [3]:
import numpy as np
import pandas as pd
import altair as alt

### <font color='darkgreen'>Dataset</font>
We also need a dataset for the examples. We will use a small sample from the [Melbourne housing dataset](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) available on Kaggle.

In [6]:
df = pd.read_csv(
    "../../datas/kaggle_melbourne_housing_snapshot/melb_data.csv",
    usecols = ['Price','Landsize','Distance','Type', 'Regionname']
)
df.head()

,Type,Price,Distance,Landsize,Regionname
0,h,1480000.0,2.5,202.0,Northern Metropolitan
1,h,1035000.0,2.5,156.0,Northern Metropolitan
2,h,1465000.0,2.5,134.0,Northern Metropolitan
3,h,850000.0,2.5,94.0,Northern Metropolitan
4,h,1600000.0,2.5,120.0,Northern Metropolitan


In [7]:
df = df[(df.Price < 3_000_000) & (df.Landsize < 1200)].sample(n=1000).reset_index(drop=True)
df.head()

,Type,Price,Distance,Landsize,Regionname
0,u,480000.0,3.3,0.0,Southern Metropolitan
1,h,2450000.0,5.4,399.0,Southern Metropolitan
2,h,940000.0,21.5,454.0,South-Eastern Metropolitan
3,t,730000.0,5.6,0.0,Southern Metropolitan
4,h,510000.0,13.5,488.0,Western Metropolitan


I have only read a small part of the original dataset. The <font color='violet'>usecols</font> parameter of the [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function allows for reading only the given columns of the csv file. I have also filtered out the outliers with regards to the `Price` and `Landsize`. Finally, a random sample of 1000 observations (<font color='brown'>i.e. rows</font>) is selected using the [sample](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) function.

## <font color='darkblue'>Introduction of Altair package</font>
**[Altair](https://altair-viz.github.io/) is a powerful library in terms of data transformations and creating interactive plots**. There are three components of interactivity.
* **Selection:** Captures interactions from the user. In other words, it selects a part of the visualization.
* **Condition:** Changes or customizes the elements based on the selection. In order to see an action, we need to attach a selection to a condition.
* **Bind:** It is a property of the selection and creates a two-way binding between a selection and input.

These concepts will be more clear as we go through the examples. Let’s first create a static scatter plot and then we will add interactive features to it.

In [8]:
alt.Chart(df).mark_circle(size=50).encode(
   x='Price',
   y='Distance',
   color='Type'
).properties(
   height=350, width=500
)

alt.Chart(...)

### <font color='darkgreen'>Altar Syntax</font>
Before starting on the interactive plots, it is better to **briefly mention the basic structure of Altair syntax**. We start by passing the data to a top-level Chart object. The data can be in the form of a Pandas dataframe or a URL string pointing to a json or csv file.

Then we describe the **type of visualization** (<font color='brown'>e.g. [mark_circle](https://altair-viz.github.io/user_guide/generated/toplevel/altair.Chart.html?highlight=mark_circle#altair.Chart.mark_circle), [mark_line](https://altair-viz.github.io/user_guide/generated/toplevel/altair.Chart.html?highlight=mark_line#altair.Chart.mark_line), and so on</font>). The <font color='blue'>encode</font> function specifies what to plot in the given dataframe. Thus, anything we write in the <font color='blue'>encode</font> function must be linked to the dataframe. Finally, we specify certain properties of the plot using the <font color='blue'>properties</font> function.

### <font color='darkgreen'>Selection & Condition</font>
Some part of the plot seems too overlapped in terms of the dots. It would look better if we can also view data points that belong to a specific type. We can achieve this in two steps. The first step is to add a selection with the `type` column and bind it to the legend:
```python
selection = alt.selection_multi(fields=['Type'], bind='legend')
```

It is not enough just to add a selection. We should somehow update the plot based on the selection. For instance, we can adjust the opacity of the data points according to the selected category by using the `condition` property with the <font color='violet'>opacity</font> parameter.

In [9]:
selection = alt.selection_multi(fields=['Type'], bind='legend')

alt.Chart(df).mark_circle(size=50).encode(
   x='Price',
   y='Distance',
   color='Type',
   opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).properties(
   height=350, width=500
).add_selection(
   selection
)

alt.Chart(...)

For the second example, we will create a scatter plot of the `Distance` and `Landsize` columns and a histogram of the `Price` column. **The histogram will be updated based on the selected area on the scatter plot**.

Since we want to select an area on the plot, we need to add a selection interval on the scatter plot:
```python
selection = alt.selection_interval()
```

This selection will will be added as a selection property to the scatter plot. For the histogram, we will use the selection as a transform filter.

In [10]:
selection = alt.selection_interval()

chart1 = alt.Chart(df).mark_circle(size=50).encode(
  x='Landsize',
  y='Distance',
  color='Type'
).properties(
  height=350, width=500
).add_selection(
  selection
)
chart2 = alt.Chart(df).mark_bar().encode(
  alt.X('Price:Q', bin=True), alt.Y('count()')
).transform_filter(
  selection
)

The `chart1` and `chart2` variables contain the scatter plot and the histogram, respectively. We can now combine and display them. **Altair is quite flexible in terms of combining multiple plots or subplots. We can even use the logical operators.**

In [11]:
chart1 | chart2

alt.HConcatChart(...)

As we can see, the histogram is updated based on the selected data points on the scatter plot. Thus, **we are able see the `Price` distribution of the selected subset.**

### <font color='darkgreen'>More</font>
**In order to better understand the concepts of the selection and condition, let’s switch the roles on the scatter plot and histogram**. We will add the selection to the histogram and use it as a transform filter on the scatter plot.

In [14]:
selection = alt.selection_interval()

chart1 = alt.Chart(df).mark_circle(size=50).encode(
   x='Landsize',
   y='Distance',
   color='Type'
).properties(
   height=350, width=500
).transform_filter(
   selection
)
chart2 = alt.Chart(df).mark_bar().encode(
   alt.X('Price:Q', bin=False), alt.Y('count()')
).add_selection(
   selection
)
chart1 | chart2

alt.HConcatChart(...)

## <font color='darkblue'>Conclusion</font>
The sky is the limit! We can create lots of different interactive plots. Altair is also quite flexible in terms of the ways to add interactive components to the visualization.

Once you have a comprehensive understanding of the elements of interactivity, you can enrich your visualizations. These elements are selection, condition, and bind. As with any other subject, practice makes perfect. The syntax may look a little bit confusing at first. However, once you understand the logic and the connections between the elements we have mentioned, creating interactive plots will become fairly easy.

## <font color='darkblue'>Supplement</font>
* [Altair - Example Gallery](https://altair-viz.github.io/gallery/index.html)